# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
# try this project here!

In [1]:
import torch
import syft as sy
hook = sy.TorchHook(torch)
import copy
from torch import nn,optim

W0628 01:03:40.806266 139714457241344 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0628 01:03:40.824520 139714457241344 deprecation_wrapper.py:119] From /home/venktesh/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



create 2 workers and a secure worker

In [2]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [3]:
secure_worker = sy.VirtualWorker(hook, id="secure_worker")
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]],requires_grad=True)

In [4]:
bob_data = data[0:2].send(bob)
bob_target = target[0:2].send(bob)
alice_data = data[2:].send(alice)
alice_target = target[2:].send(alice)

In [5]:
#initialize model
model = nn.Linear(2,1)


In [11]:
for iteration in range(10):
    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)

    bob_opt = optim.SGD(params=bob_model.parameters(), lr=0.1)
    alice_opt = optim.SGD(params=alice_model.parameters(),lr=0.1)
    for worker in range(5):
        bob_opt.zero_grad()
        bob_pred = bob_model(bob_data)
        bob_loss = ((bob_pred - bob_target)**2).sum()
        bob_loss.backward()
        bob_opt.step()
        bob_loss = bob_loss.get().data
        
        alice_opt.zero_grad()
        alice_pred = alice_model(alice_data)
        alice_loss = ((alice_pred - alice_target)**2).sum()
        alice_loss.backward()
        alice_opt.step()
        alice_loss = alice_loss.get().data
    alice_model.move(secure_worker)
    bob_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alice_model.weight.data + bob_model.weight.data) / 2).get())
        model.bias.set_(((alice_model.bias.data + bob_model.bias.data) / 2).get())
    
    print("Bob:" + str(bob_loss) + " Alice:" + str(alice_loss))

Bob:tensor(0.0013) Alice:tensor(3.4308e-05)
Bob:tensor(0.0010) Alice:tensor(2.4788e-05)
Bob:tensor(0.0008) Alice:tensor(1.8172e-05)
Bob:tensor(0.0006) Alice:tensor(1.3474e-05)
Bob:tensor(0.0005) Alice:tensor(1.0079e-05)
Bob:tensor(0.0004) Alice:tensor(7.5918e-06)
Bob:tensor(0.0003) Alice:tensor(5.7481e-06)
Bob:tensor(0.0002) Alice:tensor(4.3696e-06)
Bob:tensor(0.0002) Alice:tensor(3.3318e-06)
Bob:tensor(0.0001) Alice:tensor(2.5464e-06)


In [12]:

preds = model(data)
loss = ((preds - target) ** 2).sum()

In [13]:
print(preds)
print(target)
print(loss)

tensor([[0.0314],
        [0.0256],
        [0.9677],
        [0.9620]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0041, grad_fn=<SumBackward0>)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [1]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [2]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [3]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [4]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [5]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [6]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [14]:
x_share = (2,5,7)
import numpy as np

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [15]:
# try this project here!
Q=123789654378897

In [16]:
def encrypt(number, number_of_shares):
    shares = list()
    for i in range(number_of_shares-1):
        shares.append(np.random.randint(0,Q))
    final_share = (Q - (sum(shares)%Q))+number
    shares.append(final_share)
    return tuple(shares)

In [17]:
encrypted_value = encrypt(5,2)

In [18]:
encrypted_value

(46931108574059, 76858545804843)

In [19]:
def decrypt(shares):
    decrypted_value = sum(shares)%Q
    return decrypted_value

In [20]:
decrypt_value = decrypt(encrypted_value)
decrypt_value

5

In [24]:
def add(encrypted1, encrypted2):
    additive_result = list()
    for i in range(len(encrypted1)):
        additive_result.append((encrypted1[i]+encrypted2[i])%Q)
    return tuple(additive_result)

In [26]:
result = add(encrypt(5,2),encrypt(7,2))

In [27]:
result

(14251835169104, 109537819209805)

In [28]:
decrypt(result)

12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [25]:
BASE=10
PRECISION=4

In [26]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [27]:
encode(3.5)

35000

In [28]:
decode(35000)

3.5

In [29]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [30]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [31]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [32]:
x = x.share(bob, alice, secure_worker)

In [33]:
bob._objects

{35498656553: tensor([  10235770278698899, 1401398179551373756, 2277280072169145491,
          636965538565031298,  913795591610271305])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [34]:
y = x + x

In [35]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:23637986557 -> bob:30254176063]
	-> (Wrapper)>[PointerTensor | me:18229131498 -> alice:75856222543]
	-> (Wrapper)>[PointerTensor | me:34301722959 -> secure_worker:75419815101]
	*crypto provider: me*

In [36]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [37]:
x = th.tensor([0.1,0.2,0.3])

In [38]:
x

tensor([0.1000, 0.2000, 0.3000])

In [39]:
x = x.fix_prec()

In [40]:
x.child.child

tensor([100, 200, 300])

In [41]:
y = x + x

In [42]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [43]:
x = th.tensor([0.1, 0.2, 0.3])

In [44]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [45]:
y = x + x

In [46]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation